In [21]:
import string
from IPython.display import display, Math, Latex, Markdown

In [57]:
ref_1 = 'The cat is on the mat.'
ref_2 = 'There is a cat on the mat.'
cand_1 = 'The cat, the cat on the mat.'
cand_2 = 'The the the the the the the the.'

punctuation_list = string.punctuation
fn = lambda x: x.lower().translate(str.maketrans('', '', string.punctuation))

def extract(sentence):
    sentence = fn(sentence)
    one_gram = sentence.split()
    bi_gram = [' '.join(words) for words in zip(one_gram[::], one_gram[1::])]
    tri_gram = [' '.join(words) for words in zip(one_gram[::], one_gram[1::], one_gram[2::])]
    return one_gram, bi_gram, tri_gram

## N-gram Evaluation

### Example 1

Reference

- `{{ref_1}}` 

     $\xrightarrow[\text{}]{\text{ Preprocessing }}$ `{{fn(ref_1)}}` 
     
     $\xrightarrow[\text{}]{\text{Extract 1-gram}} $ `{{extract(ref_1)[0]}}`
     
     $\xrightarrow[\text{}]{\text{Extract 2-gram}} $ `{{extract(ref_1)[1]}}` 

     $\xrightarrow[\text{}]{\text{Extract 3-gram}} $ `{{extract(ref_1)[2]}}` 

- `{{ref_2}}` $\xrightarrow[\text{}]{\text{Preprocessing}}$ `{{fn(ref_2)}}`

Candidate

- `{{cand_1}}` $\xrightarrow[\text{}]{\text{Preprocessing}}$ `{{fn(cand_1)}}`
- `{{cand_2}}` $\xrightarrow[\text{}]{\text{Preprocessing}}$ `{{fn(cand_2)}}`

## [BLEU - Bilingual Evaluation Understudy](https://www.aclweb.org/anthology/P02-1040.pdf)

### Formula
$
\begin{align}
    \quad
        BLEU = BP \cdot exp(\sum_{n=1}^{N} w_n\log_{}{P_n}) \cr
\end{align}
$
    
$
\begin{align}
    \quad
        BP \quad\,\ = \begin{cases} 
                        1         &, \ c > r   \cr
                        e^{1-r/c} &, \ x < 0   \cr
                      \end{cases} 
\end{align}
$